In [45]:
%pip install --upgrade --quiet -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 3438-274B

 Directory of c:\Users\nirbhay\Dev\DL\Langchain\finance

14-04-2024  02:35    <DIR>          .
14-04-2024  01:35    <DIR>          ..
14-04-2024  02:44    <DIR>          .venv
14-04-2024  02:40               442 app.ipynb
14-04-2024  02:08           208,419 nb.ipynb
14-04-2024  10:04                98 requirements.txt
               3 File(s)        208,959 bytes
               3 Dir(s)  65,527,459,840 bytes free


In [82]:
import os
import streamlit as st
import pickle
import time
from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore
import tempfile


In [83]:
urls = [
    "https://en.wikipedia.org/wiki/MNIST_database",

]

In [84]:
llm = Ollama(model="mistral")


In [85]:

loader = WebBaseLoader(urls)
data = loader.load()



In [86]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(data)
len(docs)

30

In [87]:
docs[0]

Document(page_content='MNIST database - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1History\n\n\n\n\n\n\n\n2Performance\n\n\n\n\n\n\n\n3Classifiers\n\n\n\n\n\n\n\n4See also\n\n\n\n\n\n\n\n5References\n\n\n\n\n\n\n\n6Further reading\n\n\n\n\n\n\n\n7Extern

In [88]:
embeddings = OllamaEmbeddings(model="mistral")


In [89]:
from pathlib import Path
persist_path = Path("./vectore_store.pkl")
# persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")

vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persist_path,  # persist_path and serializer are optional
    serializer="parquet",
)


: 

In [32]:
query = "Fight between Ukrain"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)



Russian forces conducted ground attacks around Bakhmut and continued making tactical advances on February 8. Geolocated footage posted between February 4 and 8 confirms that Russian forces have made marginal advances north of Bakhmut near Krasna Hora and Zaliznyanske (10km north of Bakhmut), in the Stupky area of northern Bakhmut, and southwest of Bakhmut near Ivanivske.[27] Russian forces are visually confirmed to be within 2.5 km of the E40 Bakhmut-Slovyansk highway.[28] The Ukrainian General Staff also reported that Ukrainian troops repelled Russian attacks on Bakhmut itself; northeast of Bakhmut near Verkhnokamyanske (30km northeast), Fedorivka (15km northeast), Spirne (27km northeast), and Vymika (20km northeast); north of Bakhmut near Paraskoviivka (5km north) and Krasna Hora (4km north); northwest of Bakhmut near Orikhovo-Vasylivka (12km northwest) and Dubovo-Vasylivka (7km northwest); and west of Bakhmut near Ivanivske (5km west) and Chasiv Yar (10km west).[29]  The Ukrainian


In [56]:
import pickle

# Specify the file path for saving the pickle file
file_path = 'vector_store.pkl'

# Save the vector_store as a pickle file
with open(file_path, 'wb') as f:
    pickle.dump(vector_store, f)


TypeError: cannot pickle 'module' object

TypeError: isinstance expected 2 arguments, got 1

In [50]:
!pip install langchainhub

In [51]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")


from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_store.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=Ollama(model='mistral')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law an

In [77]:
question = "tell me about fights with Ukrain"
result = qa_chain({"query": question})
result

{'query': 'tell me about fights with Ukrain',
 'result': ' Russian forces have begun an offensive in Luhansk Oblast, Ukraine, conducting ground attacks around Bakhmut. Ukrainian troops are reporting repelling Russian advances in various areas including northeast and northwest of Bakhmut. The success of the Russian offensive is not yet determined.'}

In [81]:
with open("vector_Store.pkl", "rb") as f:
        vector_store = pickle.load(f)

EOFError: Ran out of input

In [ ]:
st.title("Website Research ...")
st.sidebar.title(URLs to articels)

for i in range (3):
    st.sidebar.text_input(f"URL {i+1}")

processURL_clicked = st.sidebar.button("Process URL")

if processURL_clicked:
    pass